# init

In [1]:
import comet_ml
import datatable as dt
import pandas as pd
import pyarrow.feather as feather
import pyarrow as pa
import re

from tqdm.auto import tqdm
from datatable import f, update

dt.init_styles()

# Unit test

In [12]:
comet_api = comet_ml.API()
exps = comet_api.query('amiao', 'earnings-call', 
                       (comet_ml.api.Parameter('note').startswith('MLP-2') & \
                       (comet_ml.api.Parameter('yqtr')=='2018-q1')),
                       archived=False)

In [35]:
exp = exps[0]
asset_id = [asset['assetId'] for asset in exp.get_asset_list() 
            if asset['fileName']=='y_car.feather'][0]

y = dt.Frame(feather.read_feather(pa.BufferReader(exp.get_asset(asset_id))))

# Download Comet

## Download `yt`

In [4]:
%%time
comet_api = comet_ml.API()
exps = comet_api.query('amiao', 'earnings-call', 
                       (comet_ml.api.Parameter('note').contains('MTL-30') & \
                       (comet_ml.api.Parameter('test_rmse')!=None)),
                       archived=False)

yt = dt.Frame()
print(f'N experiments: {len(exps)}')


for exp in tqdm(exps):
    note = exp.get_parameters_summary('note')['valueCurrent']
    model_name = '-'.join(re.split('-|,', note)[:2])
    try:
        asset_id = [asset['assetId'] for asset in exp.get_asset_list() 
                    if asset['fileName']=='y_car.feather'][0]
        window_size = exp.get_parameters_summary('window_size')['valueCurrent']
        yqtr = exp.get_parameters_summary('yqtr')['valueCurrent']

        y = dt.Frame(feather.read_feather(pa.BufferReader(exp.get_asset(asset_id))))
    except Exception as e:
        print(e)
        print(exp.get_name())
        print(asset_id)
        print('-----------')
        
    y[:, update(yqtr=yqtr, model_name=model_name, note=note,
                window_size=window_size)]
    
    yt.rbind(y, force=True, bynames=True)

# append new results to existing comet log
ld('yt_comet', 'old_yt', force=True)
all_yt = pd.concat([old_yt.to_pandas(), yt.to_pandas()], ignore_index=True)
all_yt = pd.DataFrame.drop_duplicates(all_yt, ignore_index=True)
all_yt = all_yt[all_yt.note.notnull()]

# all_yt = yt

sv('all_yt', 'yt_comet')

N experiments: 20


  0%|          | 0/20 [00:00<?, ?it/s]

"yt_comet.feather" (3.9 MB) loaded as "old_yt" (<1s) (2021-03-01 4:23 PM)
"all_yt" saved as "yt_comet.feather" (4.4 MB) (<1s) (2021-03-01 4:23 PM)
CPU times: user 8.09 s, sys: 2.95 ms, total: 8.1 s
Wall time: 13.6 s


## Download parameters

In [ ]:
%%time

comet_api = comet_ml.API()
exps = comet_api.query('amiao', 'earnings-call', 
                       (comet_ml.api.Parameter('note').startswith('STL-51') & \
                       (comet_ml.api.Metric('test_rmse')!=None)),
                       archived=False)

params_comet = []
for exp in exps:
    # get parameter
    log = {param['name']:param['valueCurrent'] for param in exp.get_parameters_summary()}
    
    # get metadat
    log = {**log, **exp.get_metadata()}
    
    # delete useless params
    for key in ['checkpoint_path', 'f']:
        log.pop("key", None)
    params_comet.append(log)
    

params_comet = pd.DataFrame(params_comet)

# append new results to existing comet log
old_params_comet = feather.read_feather('data/params_comet.feather')
all_params_comet = pd.concat([old_params_comet, params_comet], ignore_index=True)
all_params_comet = pd.DataFrame.drop_duplicates(all_params_comet, ignore_index=True);
feather.write_feather(all_params_comet, 'data/params_comet.feather', compression='uncompressed');

# Modify param

In [13]:
# query = ((comet_ml.api.Parameter('window_size')=='5y') &
#          (comet_ml.api.Parameter('note')=='STL-39,bsz=32,lr=3.00e-04'))

query = ((comet_ml.api.Parameter('machine')=='yu-workstation'))

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

In [9]:
for exp in exps:
    
    val = exp.get_parameters_summary('ols_rmse_norm')['valueCurrent']
    # print(window_size)
    exp.log_parameter('ols_rmse', val)

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

...

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

TypeError: list indices must be integers or slices, not str

# upload `test_start`

In [ ]:
split_df = pd.read_csv(f'data/split_dates.csv')

query = ((comet_ml.api.Parameter('test_start')==None) & \
         (comet_ml.api.Parameter('test_end')==None) & \
         (comet_ml.api.Parameter('train_start')==None) & \
         (comet_ml.api.Parameter('train_end')==None))

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

for exp in tqdm(exps):
    window = exp.get_parameters_summary('window')['valueCurrent']
    window_size = exp.get_parameters_summary('window_size')['valueCurrent']
         
    train_start = split_df.loc[(split_df.window_size==window_size) & (split_df.window==window)].train_start.to_list()[0]
    train_end = split_df.loc[(split_df.window_size==window_size) & (split_df.window==window)].train_end.to_list()[0]
    test_start = split_df.loc[(split_df.window_size==window_size) & (split_df.window==window)].test_start.to_list()[0]         
    test_end = split_df.loc[(split_df.window_size==window_size) & (split_df.window==window)].test_end.to_list()[0]
         
    # print(f'{window_size}-{test_start}')
         
    exp.log_parameter('train_start', train_start)
    exp.log_parameter('train_end', train_end)
    exp.log_parameter('test_start', test_start)
    exp.log_parameter('test_end', test_end)

# upload `OLS_rmse` 

In [19]:
bench_fr = pd.read_feather('data/bench_fr.feather')

query = (comet_ml.api.Parameter('ols_rmse_norm')!=None)

exps = comet_ml.api.API().query('amiao', 'earnings-call', query, archived=False)

for exp in tqdm(exps):
    test_start = exp.get_parameters_summary('test_start')['valueCurrent'] # char
    current_ols_rmse = float(exp.get_parameters_summary('ols_rmse_norm')['valueCurrent'])
    
    true_ols_rmse = bench_fr.loc[(bench_fr.window_size=='3y') & (bench_fr.test_start==pd.to_datetime(test_start))].test_rmse_fr_norm.to_list()[0]
    
    if current_ols_rmse != true_ols_rmse:
        exp.log_parameter('ols_rmse_norm', true_ols_rmse)

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

...

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}

{'msg': 'Saved', 'code': 200, 'data': None, 'sdkErrorCode': 0}